# Short-Term Memory Demo: Financial Advisor Assistant

## Overview

This demo shows how **short-term memory management** enables natural multi-turn conversations in a financial advisory context.

## Scenario
A financial advisor AI assistant helps clients with:
- Investment planning and recommendations
- Retirement account guidance
- Risk assessment and portfolio rebalancing
- Financial goal tracking

The assistant must remember client financial profile and preferences across the conversation without re-asking.

## Memory Components Demonstrated

1. **Conversation History** - Last-k message sliding window
2. **Session Summary** - 3-5 key facts about the client's financial situation
3. **Profile Tracking** - Structured data extraction (income, risk tolerance, goals)
4. **Context-Aware Responses** - No repetitive questions

## Setup

In [1]:
import os
import json
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass, asdict
from openai import OpenAI

# Initialize OpenAI client with Vocareum endpoint
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Environment ready!")

✅ Environment ready!


## Define Data Models

In [2]:
@dataclass
class ClientProfile:
    """Track client financial profile and preferences"""
    client_id: Optional[str] = None
    age: Optional[int] = None
    annual_income: Optional[str] = None  # e.g., "$80K-$120K"
    risk_tolerance: Optional[str] = None  # "conservative", "moderate", "aggressive"
    investment_goals: List[str] = None  # e.g., ["retirement", "home purchase", "education"]
    time_horizon: Optional[str] = None  # e.g., "5-10 years", "20+ years"
    current_investments: List[str] = None  # e.g., ["401k", "IRA", "stocks"]
    financial_concerns: List[str] = None  # e.g., ["market volatility", "inflation"]
    
    def __post_init__(self):
        if self.investment_goals is None:
            self.investment_goals = []
        if self.current_investments is None:
            self.current_investments = []
        if self.financial_concerns is None:
            self.financial_concerns = []
    
    def get_captured_info(self) -> Dict:
        """Return captured client profile"""
        info = {}
        for field, value in asdict(self).items():
            if value and (not isinstance(value, list) or len(value) > 0):
                info[field] = value
        return info

@dataclass
class ConversationTurn:
    """Single conversation turn"""
    client_input: str
    advisor_response: str
    timestamp: str

print("✅ Data models defined!")

✅ Data models defined!


## Build the Financial Advisor Assistant with Memory

In [3]:
class FinancialAdvisorAssistant:
    """Financial advisor with short-term memory management"""
    
    def __init__(self, max_history: int = 6):
        self.profile = ClientProfile()
        self.conversation_history: List[ConversationTurn] = []
        self.session_summary: List[str] = []
        self.max_history = max_history
    
    def add_turn(self, client_input: str, advisor_response: str):
        """Add conversation turn with sliding window"""
        turn = ConversationTurn(
            client_input=client_input,
            advisor_response=advisor_response,
            timestamp=datetime.now().isoformat()
        )
        self.conversation_history.append(turn)
        
        # Sliding window: keep last k turns
        if len(self.conversation_history) > self.max_history:
            self.conversation_history = self.conversation_history[-self.max_history:]
    
    def update_session_summary(self, client_input: str):
        """Update running summary of client's financial situation and context"""
        summary_prompt = f"""Current session summary about the client:
{chr(10).join('- ' + fact for fact in self.session_summary) if self.session_summary else 'None yet'}

Latest client input: "{client_input}"

Update the summary to include new information. Keep 3-5 key facts about:
- Client's financial goals and timeline
- Risk tolerance and investment preferences
- Current financial situation
- Key concerns or priorities

Return only the bullet points, one per line, without bullet symbols."""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": summary_prompt}],
                temperature=0.1,
                max_tokens=150
            )
            
            new_summary = [
                line.strip() 
                for line in response.choices[0].message.content.strip().split('\n') 
                if line.strip()
            ]
            self.session_summary = new_summary[-5:]
        except Exception as e:
            print(f"Error updating summary: {e}")
    
    def extract_profile(self, client_input: str):
        """Extract structured profile information from client input"""
        extraction_prompt = f"""Extract client financial profile from: "{client_input}"

Current profile:
{json.dumps(self.profile.get_captured_info(), indent=2)}

Extract and return ONLY new information in valid JSON:
{{
    "client_id": "client ID or null",
    "age": number or null,
    "annual_income": "income range or null",
    "risk_tolerance": "conservative|moderate|aggressive or null",
    "investment_goals": ["goals"] or null,
    "time_horizon": "time horizon or null",
    "current_investments": ["investment types"] or null,
    "financial_concerns": ["concerns"] or null
}}

Return valid JSON only."""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.1,
                max_tokens=200
            )
            
            content = response.choices[0].message.content.strip()
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()
            
            extracted = json.loads(content)
            
            # Update profile
            for field, value in extracted.items():
                if value and value != "null" and hasattr(self.profile, field):
                    if isinstance(value, list) and len(value) > 0:
                        # Append to list fields
                        current = getattr(self.profile, field) or []
                        setattr(self.profile, field, list(set(current + value)))
                    elif not isinstance(value, list):
                        setattr(self.profile, field, value)
        
        except Exception as e:
            print(f"Error extracting profile: {e}")
    
    def generate_response(self, client_input: str) -> str:
        """Generate contextual response using memory"""
        
        # Build memory context
        memory_context = ""
        if self.session_summary:
            memory_context = f"""What you know about the client:
{chr(10).join('- ' + fact for fact in self.session_summary)}
"""
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "Recent conversation:\n"
            for turn in self.conversation_history[-3:]:
                conversation_context += f"Client: {turn.client_input}\nYou: {turn.advisor_response}\n\n"
        
        captured = self.profile.get_captured_info()
        
        response_prompt = f"""You are a professional financial advisor assistant.

{memory_context}

{conversation_context}

Client profile captured:
{json.dumps(captured, indent=2) if captured else 'None yet'}

Client just said: "{client_input}"

Guidelines:
1. Be professional, trustworthy, and informative
2. Reference what you already know from the session summary
3. NEVER ask for information you already have
4. If they mention financial details, acknowledge and use them
5. Make personalized recommendations based on their profile
6. Keep responses concise (2-3 sentences)
7. Provide sound financial guidance

Respond naturally as a financial advisor:"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": response_prompt}],
                temperature=0.3,
                max_tokens=200
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return "I apologize, I'm experiencing technical difficulties. How may I assist you with your financial planning?"
    
    def process_input(self, client_input: str) -> str:
        """Main processing: update memory, extract info, respond"""
        # Update session summary
        self.update_session_summary(client_input)
        
        # Extract profile
        self.extract_profile(client_input)
        
        # Generate response
        response = self.generate_response(client_input)
        
        # Add to history
        self.add_turn(client_input, response)
        
        return response

print("✅ FinancialAdvisorAssistant defined!")

✅ FinancialAdvisorAssistant defined!


## Demo: Multi-Turn Financial Advisory Conversation

In [4]:
def run_advisor_demo():
    """Demonstrate memory management in financial advisory scenario"""
    
    assistant = FinancialAdvisorAssistant(max_history=4)
    
    conversation = [
        "Hi, I'm 35 years old and want to start investing for retirement. I have about $50K saved up.",
        "I'm pretty nervous about the stock market. I'd say I'm more on the conservative side when it comes to risk.",
        "My main goal is to have enough for retirement in 30 years, but I also want to buy a house in the next 5 years.",
        "That makes sense. What about my current 401k? I have about $30K in there already. Should I max it out?",
        "Good point about the employer match. I'm currently contributing 3% and they match 6%. I should increase that.",
        "One more thing - I'm worried about inflation eating into my savings. What should I do about that?",
        "Thanks! Can you summarize the key recommendations we discussed today for my retirement planning?"
    ]
    
    print("💼 FINANCIAL ADVISOR CONVERSATION DEMO")
    print("="*60)
    print()
    
    for i, client_input in enumerate(conversation, 1):
        print(f"👤 CLIENT (Turn {i}): {client_input}")
        
        response = assistant.process_input(client_input)
        print(f"💰 ADVISOR: {response}")
        print()
        
        # Show memory state periodically
        if i % 2 == 0:
            print("🧠 MEMORY STATE:")
            print(f"   📝 Summary: {assistant.session_summary}")
            print(f"   💾 Profile: {list(assistant.profile.get_captured_info().keys())}")
            print(f"   📚 History: {len(assistant.conversation_history)} turns")
            print()
    
    return assistant

# Run the demo
demo_assistant = run_advisor_demo()

💼 FINANCIAL ADVISOR CONVERSATION DEMO

👤 CLIENT (Turn 1): Hi, I'm 35 years old and want to start investing for retirement. I have about $50K saved up.
💰 ADVISOR: Hello! It's great to hear that you're ready to start investing for retirement. With your $50K savings, we can explore various investment options that align with your goals. Given your age, you have a long investment horizon, which typically allows for a more aggressive strategy focused on growth. We can discuss options like a diversified portfolio of stocks, index funds, or even a target-date retirement fund that adjusts as you approach retirement. Would you like to explore these options further?

👤 CLIENT (Turn 2): I'm pretty nervous about the stock market. I'd say I'm more on the conservative side when it comes to risk.
💰 ADVISOR: Hello! It's great to hear that you're ready to start investing for retirement. With your $50K savings, we can explore various investment options that align with your goals. Given your age, you have

## Comparison: Without Memory Management

In [5]:
print("\n" + "="*60)
print("❌ PROBLEM: Without Memory Management")
print("="*60)

def broken_advisor_interaction():
    """Show problems without memory"""
    print("\n🔴 Conversation without memory management...")
    print()
    print("Turn 1:")
    print("  👤 Client: 'I'm 35 and have a conservative risk tolerance'")
    print("  💰 Advisor: 'Great! I can help with that.'")
    print()
    print("Turn 2:")
    print("  👤 Client: 'What investments do you recommend?'")
    print("  💰 Advisor: 'What's your age and risk tolerance?'")
    print("  ⚠️  FORGOT: Client just shared age and risk tolerance!")
    print()
    print("Turn 3:")
    print("  👤 Client: 'What about my 401k strategy?'")
    print("  💰 Advisor: 'Do you have any existing retirement accounts?'")
    print("  ⚠️  IRRELEVANT: Already discussing 401k!")
    print()
    print("Turn 4:")
    print("  👤 Client: 'Should I increase my contribution?'")
    print("  ⚠️  LOST: Client's age and timeline")
    print("  ⚠️  LOST: Risk tolerance")
    print("  ⚠️  LOST: Current 401k balance")
    print("  ⚠️  LOST: All financial goals discussed")
    print("  ⚠️  No continuity - frustrating experience!")

broken_advisor_interaction()

print("\n" + "="*60)
print("✅ SOLUTION: With Memory Management")
print("="*60)
print("✓ Remembers client profile and goals across turns")
print("✓ No repetitive questions about age, risk tolerance, etc.")
print("✓ Personalized recommendations based on full context")
print("✓ Natural conversation flow")
print("✓ Professional advisory experience")


❌ PROBLEM: Without Memory Management

🔴 Conversation without memory management...

Turn 1:
  👤 Client: 'I'm 35 and have a conservative risk tolerance'
  💰 Advisor: 'Great! I can help with that.'

Turn 2:
  👤 Client: 'What investments do you recommend?'
  💰 Advisor: 'What's your age and risk tolerance?'
  ⚠️  FORGOT: Client just shared age and risk tolerance!

Turn 3:
  👤 Client: 'What about my 401k strategy?'
  💰 Advisor: 'Do you have any existing retirement accounts?'
  ⚠️  IRRELEVANT: Already discussing 401k!

Turn 4:
  👤 Client: 'Should I increase my contribution?'
  ⚠️  LOST: Client's age and timeline
  ⚠️  LOST: Risk tolerance
  ⚠️  LOST: Current 401k balance
  ⚠️  LOST: All financial goals discussed
  ⚠️  No continuity - frustrating experience!

✅ SOLUTION: With Memory Management
✓ Remembers client profile and goals across turns
✓ No repetitive questions about age, risk tolerance, etc.
✓ Personalized recommendations based on full context
✓ Natural conversation flow
✓ Professiona

## Memory Analysis

In [6]:
print("🔍 DETAILED MEMORY ANALYSIS")
print("="*50)
print()

print("1️⃣ SESSION SUMMARY:")
for i, fact in enumerate(demo_assistant.session_summary, 1):
    print(f"   {i}. {fact}")
print()

print("2️⃣ CONVERSATION HISTORY (Sliding Window):")
for i, turn in enumerate(demo_assistant.conversation_history, 1):
    print(f"   Turn {i}:")
    print(f"      👤: {turn.client_input[:50]}...")
    print(f"      💰: {turn.advisor_response[:50]}...")
print()

print("3️⃣ EXTRACTED CLIENT PROFILE:")
for key, value in demo_assistant.profile.get_captured_info().items():
    print(f"   {key}: {value}")
print()

print("4️⃣ MEMORY METRICS:")
print(f"   📊 Conversation turns: {len(demo_assistant.conversation_history)}")
print(f"   🎯 Profile fields captured: {len(demo_assistant.profile.get_captured_info())}")
print(f"   🧠 Summary facts: {len(demo_assistant.session_summary)}")
print(f"   📝 Max history size: {demo_assistant.max_history}")

🔍 DETAILED MEMORY ANALYSIS

1️⃣ SESSION SUMMARY:
   1. Client is 35 years old and aims to have enough for retirement in 30 years.
   2. Client has approximately $50K saved for retirement, including $30K in a current 401k, and plans to buy a house in the next 5 years.
   3. Client has a conservative risk tolerance and is nervous about the stock market, preferring safer investment options.
   4. Client is currently contributing 3% to their 401k with a 6% employer match and is considering increasing their contribution to boost retirement savings.
   5. Client is concerned about inflation impacting their savings and is seeking strategies to mitigate this risk.

2️⃣ CONVERSATION HISTORY (Sliding Window):
   Turn 1:
      👤: That makes sense. What about my current 401k? I ha...
      💰: It's great to hear you're considering maximizing y...
   Turn 2:
      👤: Good point about the employer match. I'm currently...
      💰: Absolutely, increasing your contribution to take f...
   Turn 3:
      

## Key Insights

### Memory Management Pattern

This financial advisor assistant demonstrates the short-term memory pattern:

**3 Memory Components**:
1. ✅ **Conversation History** - Sliding window of recent turns
2. ✅ **Session Summary** - 3-5 key facts about the client's financial situation
3. ✅ **Structured Data** - Extracted client profile (age, risk tolerance, goals)

**Key Benefits**:
- **No Repetition**: Never re-asks for known information (age, risk tolerance, etc.)
- **Personalization**: Recommendations based on complete financial profile
- **Context Continuity**: Maintains conversation thread across topics
- **Efficient**: Bounded memory prevents token overflow

### Financial Services Applications

This pattern is essential for financial AI agents:
- 💰 **Investment Advisory** (this demo) - Portfolio recommendations and retirement planning
- 💳 **Customer Dispute Intake** - Gathering transaction details and resolution preferences
- 🏦 **Loan Applications** - Collecting income, credit, and property information
- 📊 **Financial Planning** - Understanding goals, timeline, and risk tolerance
- 🔒 **Fraud Detection** - Tracking patterns and customer verification
- 💼 **Wealth Management** - Long-term client relationship management

### Production Considerations

Real financial services systems need:
- **Compliance**: Ensure memory practices meet regulatory requirements (GDPR, CCPA)
- **Security**: Encrypt sensitive financial data in memory and storage
- **Audit Trails**: Log all interactions for compliance and dispute resolution
- **Persistence**: Store client profiles in secure databases
- **Privacy**: Clear PII appropriately and follow data retention policies
- **Multi-Session**: Link memory across different conversation sessions

But the **core 3-component pattern remains consistent**!